In [ ]:
### ESTO VA ANTES DE 02c!!!!

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import hashlib

import dask
import xarray as xr
import xrspatial
from dask.diagnostics import ProgressBar
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt
import seaborn as sns

PATH = "Z:\Laboral\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


In [2]:
# floods = pd.read_csv(rf"{DATA_RAW}\Floods\GloFAS_floods.csv")

def load_population_data(bounds=None):
    
    # Select all files in GPW folder
    gpw_path = r"Z:\WB Data\Gridded Population of The World (GPWv4)"
    files = os.listdir(gpw_path)
    files = [f for f in files if f.endswith(".tif")]
    
    # Compile into a single dataset
    dss = []
    for f in files:
        ds = xr.open_dataset(os.path.join(gpw_path, f), chunks={"x": 100, "y": 100})
        ds["year"] = int(f.split("_")[5])
        ds = ds.set_coords('year')
        dss += [ds]
    population = xr.concat(dss, dim="year")    
    
    # Filter if bounds are provided
    if bounds is not None:
        population = population.sel(
            x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
        )
        
    # Clean band dimension
    population = population.sel(band=1).drop_vars(["band"])
    
    return population

def load_precipitation_data():
    era5 = xr.open_dataset(
        rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc",
        chunks={"latitude": 100, "longitude": 100},
    )
    era5 = era5.rename({"latitude": "y", "longitude": "x"})
    return

def load_WB_country_data(drop_adm2_na=True):
    WB_country = gpd.read_file(rf"{DATA_RAW}\world_bank_adm2.zip")
    
    WB_country = WB_country.dissolve(by=["ADM2_CODE", "ADM1_CODE", "ADM0_CODE"]).reset_index()
    # Assign nan when ADM2 is not available 
    WB_country.loc[WB_country.ADM2_NAME == "Administrative unit not available", "ADM2_CODE"] = (
        np.nan
    )
    
    # Create ADM_LAST variable: ADM2_NAME if available, else ADM1_NAME
    WB_country["ADM_LAST"] = WB_country.ADM2_NAME
    WB_country.loc[WB_country.ADM_LAST.isnull(), "ADM_LAST"] = WB_country.ADM1_NAME
    
    if drop_adm2_na:
        # Drop rows with missing ADM2_CODE    
        WB_country = WB_country.dropna(subset=["ADM2_CODE"])
        
    # Create ID
    WB_country["ID"] = WB_country.groupby(["ADM2_CODE", "ADM1_CODE", "ADM0_CODE"]).ngroup()
    assert WB_country.ID.nunique() == WB_country.shape[0], "ID is not unique!, there's some bug in the code..."
    return WB_country


def rasterize_shape_like_dataset(shape, dataset):
    print("Rasterizing shape...")
    raster = make_geocube(
        vector_data=shape,
        like=dataset,
    )
    # For some reason, like option is not working, so I have to manually add x and y
    assert (raster["x"].shape == dataset["x"].shape)
    assert (raster["y"].shape == dataset["y"].shape)
    raster["x"] = dataset["x"]
    raster["y"] = dataset["y"]
    raster = raster.drop_vars(["spatial_ref"])
    raster = raster.chunk({"x": 100, "y": 100})
    print("Done!")
    return raster

def compute_zonal_stats(dataset, shape, value_var, groupby_var, gridded_groups=None, stats_funcs=["sum"], delayed=True):
    
    # Rasterize shape
    if gridded_groups is None:
        gridded_groups = rasterize_shape_like_dataset(shape[[groupby_var, "geometry"]], dataset)

    # Compute zonal stats  
    assert gridded_groups.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."
    assert dataset.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."

    print("Setting up zonal stats...")
    pop_by_adm = xrspatial.zonal.stats(gridded_groups[groupby_var], dataset[value_var], stats_funcs=stats_funcs)
    print("Done! Computing zonal stats...")    
    if delayed:
        return pop_by_adm
    
    with ProgressBar():
        pop_by_adm = pop_by_adm.compute()
    
    # Format zonal_stats dataframe
    pop_by_adm = pop_by_adm.rename(columns={
        "sum": value_var,
        "mean": f"{value_var}_mean",
        "zone": groupby_var,
    })
    
    result = (
        shape[[groupby_var, "geometry"]]
        .merge(pop_by_adm, on=groupby_var)
    )
    return result 

def compute_zonal_stats_over_time(dataset, shape, value_var, groupby_var, population_data=None, gridded_groups=None, stats_funcs=["mean"], delayed=True):
    import warnings
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    
    # Rasterize shape
    if gridded_groups is None:
        gridded_groups = rasterize_shape_like_dataset(shape[[groupby_var, "geometry"]], dataset)

    # Compute zonal stats  
    assert gridded_groups.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."
    assert dataset.chunks is not None, "Please, chunk the dataset before computing zonal stats! (e.g. dataset.chunk({'x': 100, 'y': 100})). Otherwise, you will get a MemoryError."
    assert "year" in dataset.dims, "Please, add a 'year' dimension to the dataset before computing zonal stats! (e.g. dataset = dataset.assign_coords(year=dataset.time.dt.year))."
 
    print("Setting up zonal stats...")
    tasks = []
    for year in tqdm(dataset["year"].values):
        dataset_year = dataset.sel(year=year).drop_vars("year")
        if population_data is not None:
            dataset_year = dataset_year * population_data.sel(year=year, method="nearest").drop_vars("year")
        else:
            dataset_year[value_var] = dataset_year[value_var].astype("float32")

        tasks += [xrspatial.zonal.stats(zones=gridded_groups[groupby_var], values=dataset_year[value_var], stats_funcs=stats_funcs)]
    if delayed:
        return tasks

    print("Done! Computing zonal stats...")        
    with ProgressBar():
        result = dask.compute(*tasks)
        
    return result 

def compile_zonal_stats_over_time(tasks_results, shape, groupby_var, value_var):
    
    # Compile results into a single df
    out_dict = {year: data.set_index("zone") for year, data in zip(range(1970,2021), tasks_results)}
    df = pd.concat(out_dict)
    df = df.reset_index()
    df = df.rename(columns={"level_0":"year"})
    
    # Format zonal_stats dataframe
    df = df.rename(columns={
        "sum": value_var,
        "mean": value_var,
        "zone": groupby_var,
    })
    
    result = (
        shape[[groupby_var, "geometry"]]
        .merge(df, on=groupby_var)
    )
    return result

def process_era5_data():
   
    # Load ERA5 data
    
    # Create droughts dummies
    
    # Annualize series
    
    return

# Genera grilla con WB Country

In [22]:
### ESTO VA ANTES DE 02c!!!!
WB_country = load_WB_country_data(drop_adm2_na=True)
population = load_population_data(bounds=WB_country.total_bounds)

# Rasterize WB_country
WB_country_grid = rasterize_shape_like_dataset(
    WB_country[["ID", "geometry"]], 
    population
)

WB_country_path = rf"E:\WB_country_grid.nc"
print("Saving WB_country_grid...")
with ProgressBar():
    WB_country_grid.to_netcdf(WB_country_path)
        
WB_country[["ID", "ADM2_CODE", "ADM2_NAME", "ADM1_CODE", "ADM1_NAME", "ADM0_CODE", "ADM0_NAME", "geometry"]].to_feather(rf"E:\client_v2_data\WB_country_IDs.feather")

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:159: RuntimeWarning: 'ee' fails while gu

# GENERA Base de shocks, sin interpolar

In [17]:
droughts_path = rf"{DATA_OUT}\ERA5_droughts_1970-2021.nc"
# if not os.path.exists(droughts_path):
print("Preparing droughts dataset...")
# Genera base de sequías
era5 = xr.open_dataset(rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc", chunks={"latitude": 500, "longitude": 500})

Preparing droughts dataset...


In [20]:
droughts_path = rf"E:\client_v2_data\ERA5_droughts_1970-2021.nc"
# if not os.path.exists(droughts_path):
print("Preparing droughts dataset...")
# Genera base de sequías
era5 = xr.open_dataset(rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc", chunks=)
# Corrije la dimensión x, que va de 0 a 360
era5 = era5.rename({'latitude': 'y', 'longitude': 'x'})
era5['x'] = np.linspace(-180, 180, era5.x.size)

# era5 = era5.chunk({'time': 5})

# Calcula las sequías anuales
spi_yearly = era5.groupby("time.year").min()
with ProgressBar():
    spi_yearly.to_netcdf(rf"E:\client_v2_data\ERA5_yearly_1970-2021_SPI-SPEI.nc")

# droughts_resamp = droughts_yearly.reindex(y=WB_country_grid.y, x=WB_country_grid.x, method="nearest")

# droughts_resamp = droughts_yearly.interp_like(WB_country_grid, method="nearest")
# droughts_resamp = droughts_resamp.chunk({'x': 100, 'y': 100, 'year': 5})
#     print("Saving droughts...")
#     with ProgressBar():
#         droughts_resamp.to_netcdf(droughts_path)
#     print("Done!")
# else:
#     droughts_resamp = xr.open_dataset(droughts_path)

Preparing droughts dataset...
[########################################] | 100% Completed | 2hr 1ms


In [28]:
spi_yearly = xr.open_dataset(rf"E:\client_v2_data\ERA5_yearly_1970-2021_SPI-SPEI.nc")
spi_yearly

<xarray.Dataset> Size: 40GB
Dimensions:  (year: 51, y: 1799, x: 3600)
Coordinates:
  * x        (x) float64 29kB -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * y        (y) float32 7kB 89.9 89.8 89.7 89.6 ... -89.6 -89.7 -89.8 -89.9
  * year     (year) int64 408B 1970 1971 1972 1973 1974 ... 2017 2018 2019 2020
Data variables: (12/15)
    d2m      (year, y, x) float64 3GB ...
    t2m      (year, y, x) float64 3GB ...
    sp       (year, y, x) float64 3GB ...
    tp       (year, y, x) float64 3GB ...
    PET      (y, x, year) float64 3GB ...
    SPI-1    (y, x, year) float64 3GB ...
    ...       ...
    SPI-6    (y, x, year) float64 3GB ...
    SPEI-6   (y, x, year) float64 3GB ...
    SPI-12   (y, x, year) float64 3GB ...
    SPEI-12  (y, x, year) float64 3GB ...
    SPI-24   (y, x, year) float64 3GB ...
    SPEI-24  (y, x, year) float64 3GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-11 19:34:49 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [29]:
spi_yearly = xr.open_dataset(rf"E:\client_v2_data\ERA5_yearly_1970-2021_SPI-SPEI.nc", chunks={"x": 900, "y": 1800})

spi_spei_vars = [var for var in spi_yearly.data_vars if "-" in var]
for var in spi_spei_vars:
    for threshold_str in ["1_0", "1_5", "2_0", "2_5"]:
        threshold = float(threshold_str.replace("_", "."))
        spi_yearly[f"drought_{var}_{threshold_str}"] = (spi_yearly[var] < -threshold).astype("bool")

spi_yearly = spi_yearly[[var for var in spi_yearly.data_vars if "drought" in var]]
with ProgressBar():
    spi_yearly.to_netcdf(droughts_path)

[########################################] | 100% Completed | 313.82 s
